In [141]:
import os

import cv2
from dask import delayed
import numpy as np
from matplotlib import pyplot as plt
import sklearn
from sklearn.linear_model import Perceptron, RidgeClassifier

from sympy import Symbol
import tqdm

In [2]:
FACE_SIZE = 19

In [3]:
def compute_intg_image(img):
    """
    Computes matrix for quick computation of integrals.
    """
    # Create a matrix of zeros with the same dimensions as the input image
    intg_img = np.zeros((img.shape[0]+1, img.shape[1]+1) , dtype=np.int64)
    
    # Iterate over each pixel in the input image
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            # Calculate the sum of all pixels above and to the left of the current pixel
            intg_img[i+1, j+1] = intg_img[i, j+1] + intg_img[i+1, j] - intg_img[i, j] + img[i, j]

    # Return the computed integral image
    return intg_img

def get_rect(intg_img, si, sj, ei, ej):
    """
    Computes integral in rect by top left corner to bottom right
    """
    # Calculate the sum of all pixels within the specified rectangular region
    result = intg_img[ei+1, ej+1] - intg_img[si, ej+1] - intg_img[ei+1, sj] + intg_img[si, sj]
    
    # Return the computed sum of pixels
    return result

In [63]:
def generate_features_from_filter(h, w, filt_type):
    gen_features = []
    for i in range(FACE_SIZE-h+1):
        for j in range(FACE_SIZE-w+1):
            gen_features.append((h,w, filt_type, i, j))
    return gen_features

def generate_haar_filters():
    # (r, c, is_vert, i, j)
    haar_features= []
    for i in range(1, FACE_SIZE+1):
        for j in range(1, FACE_SIZE+1):
            if i % 2 == 0: #vert-seg
                filter_type = 0
                haar_features += generate_features_from_filter(i, j, filter_type)
            if j % 2 == 0: #hori-seg
                filter_type = 1
                haar_features += generate_features_from_filter(i, j, filter_type)
            if j % 3 == 0: #hori-seg
                filter_type = 2
                haar_features += generate_features_from_filter(i, j, filter_type)
            if i % 3 == 0: #vert-seg
                filter_type = 3
                haar_features += generate_features_from_filter(i, j, filter_type)

    return haar_features

def compute_haar_feature(intg_img, haar_feature):
    h, w, filt_type, i, j = haar_feature
    result = 0
    if filt_type == 0:
        result = 2 * ( intg_img[i+h, j+w] - 2*intg_img[i+h//2, j+w] - intg_img[i+h, j] + 2*intg_img[i+h//2, j] + intg_img[i, j+w] - intg_img[i, j]) / (h*w)
    elif filt_type == 1:
        result = 2*(2*intg_img[h + i, j + w//2] - intg_img[h + i, j + w] - intg_img[h + i, j] - 2*intg_img[i, j + w//2] + intg_img[i, j + w] + intg_img[i, j]) / (h*w)
    elif filt_type == 2:
        result = (3*intg_img[h + i, j + 2*w//3] - 3*intg_img[h + i, j + w//3] - intg_img[h + i, j + w] + intg_img[h + i, j] - 3*intg_img[i, j + 2*w//3] + 3*intg_img[i, j + w//3] + intg_img[i, j + w] - intg_img[i, j]) / (h*w) * 1.5
    elif filt_type == 3:
        result = (-intg_img[h + i, j + w] + intg_img[h + i, j] + 3*intg_img[h//3 + i, j + w] - 3*intg_img[h//3 + i, j] - intg_img[i + 2*h//3, j + w] + intg_img[i + 2*h//3, j] - intg_img[i, j + w] + intg_img[i, j])/(h*w)*1.5
    return result

def compute_haar_coef(intg_img, haar_filter_set):
    return np.hstack([compute_haar_feature(intg_img, filt) for filt in haar_filter_set])


In [148]:
def load_folder(folder):
    imgs = []
    for file in os.listdir(folder):
        if ".pgm" in file:
            imgs.append(cv2.imread(folder+"/"+file,-1))
    return imgs

def load_data(folder):
    train_x_true = load_folder(folder+"/face")[:1000]
    train_x_false = load_folder(folder+"/non-face")[:3000]

    train_y_true = [1] * len(train_x_true)
    train_y_false = [0] * len(train_x_false)

    train_x = train_x_true + train_x_false
    train_y = train_y_true + train_y_false
    train_x = [compute_intg_image(img) for img in train_x]
    return train_x, train_y

In [149]:
train_x, train_y = load_data("train")

In [163]:
def Adaboost(train_x, train_y, d=1, T=100):
    face_count = 0
    non_face_count = 0
    for i, label in enumerate(train_y):
        if label:
            face_count += 1
        else:
            non_face_count += 1
    print(face_count, non_face_count)
    w = np.array([1/face_count if train_y[i] else 1/non_face_count for i in range(len(train_y))])

    classifiers = []
    feature_sel = []
    betas = []
    model_threshold = 0
    def select_features(x_data, x_sel):
        return [[x[f] for f in x_sel] for x in x_data]
    predictions = np.zeros(train_y.shape)
    pbar = tqdm.tqdm(range(T))
    for t in pbar:
        w = w / np.sum(w)
        while True: 
            model = RidgeClassifier()
            m_sel = np.random.permutation(len(train_x[0]))[:d]
            m_train_x = select_features(train_x, m_sel)
            model.fit(m_train_x, train_y, sample_weight=w)
            e = np.abs(train_y - model.predict(m_train_x))
            
            beta = np.sum(w * e)
            if beta < 0.5:
                
                feature_sel.append(m_sel)
                w = w * np.power(beta, 1-e)
                classifiers.append(model)
                betas.append(beta+1e-6)
                break
        alpha = -np.log(betas)
        predictions = predictions + alpha[-1]*np.array(classifiers[-1].predict(select_features(train_x, feature_sel[-1])))
        model_threshold = np.min(predictions[train_y==1])
        miss_classification = np.sum((predictions >= model_threshold) != train_y)
        pbar.set_description(f'Error: {miss_classification}')
        if miss_classification == 0:
            break  
        
    
    def model(x):
        preds = np.sum( np.array([c.predict(select_features(x, f_sel)) for c, f_sel in zip(classifiers, feature_sel)]) * alpha.reshape(-1,1), axis=0)
        return preds >= model_threshold
    return model
    

In [164]:
def train_classifier(train_x, train_y):
    final_cascade = []
    x = train_x
    y = np.array(train_y)
    d_size = 1
    while True:
        model=Adaboost(x,y,  200,20)
        y_pred = model(x)
        
        pos_sample = np.argwhere(y_pred==True).flatten()
        new_x = [x[i] for i in pos_sample]
        new_y = [y[i] for i in pos_sample]
        d_size += 1
        final_cascade.append(model)
        if( (y_pred==y).all() ):
            break

        x = new_x
        y = np.array(new_y)
    return final_cascade

In [165]:
haar_filters = generate_haar_filters()
np.random.shuffle(haar_filters)

In [166]:
final_cascade = []
final_filters = []
stage_filters = []
x = train_x
y = np.array(train_y)
d_size = 1
filt_size = 100
while True:
    stage_filters += haar_filters[:filt_size]
    haar_filters = haar_filters[filt_size+1:]
    filt_size *= 2
    filt_size = min(filt_size, len(haar_filters))
    print(f"Stage: {len(final_cascade)+1} - {len(stage_filters)}")
    f_x = [compute_haar_coef(img, stage_filters) for img in x]
    print("Train start")
    model=Adaboost(f_x, y, len(stage_filters)//2, 100)
    print("Done")
    y_pred = model(f_x)

    pos_sample = np.argwhere(y_pred==True).flatten()
    new_x = [x[i] for i in pos_sample]
    new_y = [y[i] for i in pos_sample]
    d_size += 1
    final_cascade.append(model)
    final_filters.append(len(stage_filters))
    if( (y_pred==y).all() ):
        break
    x = new_x
    y = np.array(new_y)


Stage: 1 - 100
Train start
1000 3000


Error: 816: 100%|█████████████████████████████| 100/100 [00:13<00:00,  7.65it/s]


Done
Stage: 2 - 300
Train start
1000 816


Error: 46: 100%|██████████████████████████████| 100/100 [00:19<00:00,  5.05it/s]


Done
Stage: 3 - 700
Train start
1000 46


Error: 0:   3%|▉                                | 3/100 [00:00<00:31,  3.04it/s]


Done


In [160]:
 def check_img(img):
    intg_img = compute_intg_image(img)
    filt_size = 100
    t_filt = 100
    for i in range(len(final_cascade)):
        s_f = stage_filters[:t_filt]
        filt_size *= 2
        t_filt += filt_size
        filt_size = min(filt_size, len(haar_filters))
        f_x = compute_haar_coef(intg_img, s_f)
        if not(final_cascade[i]([f_x])):
            return False
    return True

In [161]:
check_img(load_folder('data')[1])

False

In [167]:

  
# define a video capture object
vid = cv2.VideoCapture(0)

while(True):
      
    # Capture the video frame
    # by frame
    ret, frame = vid.read()
    img = cv2.resize(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), (200,200))
    cimg = img.copy()
    k_size = 80
    for i in range(0,img.shape[0]-k_size+1, k_size//2):
        for j in range(0,img.shape[1]-k_size+1, k_size//2):
            if(check_img(cv2.resize(img[i:i+k_size, j:j+k_size], (19,19))) ):
                cv2.rectangle(cimg, (j,i), (j+k_size, i+k_size), 255, 2)
                cv2.imshow('face',cv2.resize(img[i:i+k_size, j:j+k_size], (50,50)) )
                cv2.waitKey(1)
            cv2.imshow('frame', cimg)
            cv2.waitKey(5)
# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [168]:
vid.release()